### Create new autoscale container, using the built-in .NET SDK ###

In [ ]:
#r "nuget: Microsoft.Azure.Cosmos, 3.9.1"

In [ ]:
using Microsoft.Azure.Cosmos;

CosmosClient cosmosClient = new CosmosClient(Cosmos.Endpoint, Cosmos.Key);
ContainerProperties containerProperties = new ContainerProperties("AutoscaleContainer", "/id");

var autoscaleThroughput = ThroughputProperties.CreateAutoscaleThroughput(4000);

Database database = await cosmosClient.CreateDatabaseIfNotExistsAsync("Demo");
Container container = await database.CreateContainerIfNotExistsAsync(containerProperties, autoscaleThroughput);

Display.AsMarkdown($"Created autoscale container that will scale between {0.1 * autoscaleThroughput.AutoscaleMaxThroughput} to {autoscaleThroughput.AutoscaleMaxThroughput} RU/s, based on usage.");

### Setup ###

In [ ]:
using System.Threading;
using System.Threading.Tasks;
using System.Diagnostics;
using System;
using System.Collections;
using System.Collections.Generic;
using System.Collections.Concurrent;

var taskCount = 2;
var pendingTaskCount = taskCount;

var numberOfItemsToInsertPerTask = 1000;
var delayBetweenOperationsInMs = 0;
var delayBetweenLogOutputInSeconds = 1;

long documentsInserted;

ConcurrentDictionary<int, double> requestUnitsConsumed = new ConcurrentDictionary<int, double>();

class TestItem
{
    public string id { get; set; }
    
    public string val { get; set; }

}

### Helper method to print out RU/s usage

In [ ]:
public async Task LogOutputStatsSimple() {
    long lastCount = 0;
    double currentRequestUnits = 0;
    double lastRequestUnits = 0;
    double lastDocumentCount = 0;
    double lastSeconds = 0;

    Stopwatch watch = new Stopwatch(); //start counting - each task starts from 0
    watch.Start();
    
    await Task.Delay(TimeSpan.FromSeconds(delayBetweenLogOutputInSeconds)); //wait 1 second

    while (pendingTaskCount >0) {
        double currentSeconds = watch.Elapsed.TotalSeconds;

        await Task.Delay(TimeSpan.FromSeconds(delayBetweenLogOutputInSeconds)); //wait 1 second

        currentRequestUnits = 0;
        foreach (int taskId in requestUnitsConsumed.Keys) // Sum up the total RU/s consumed across all tasks
        {
            currentRequestUnits += requestUnitsConsumed[taskId];
        }
        
        var timeElapsed = currentSeconds - lastSeconds;
        double currentRuPerSecond =  Math.Round((currentRequestUnits - lastRequestUnits) / timeElapsed);
        double currentDocumentCount = documentsInserted;
        double currentWritesPerSecond = Math.Round((currentDocumentCount - lastDocumentCount) / timeElapsed);

        Display.AsMarkdown($"Inserted {currentDocumentCount - lastDocumentCount} documents at {currentRuPerSecond} RU/s. Current writes/s is: {currentWritesPerSecond} writes/s");

        lastSeconds = currentSeconds;
        lastRequestUnits = currentRequestUnits;
        lastDocumentCount = currentDocumentCount;
        
    }
    Display.AsMarkdown($"Finished all inserts. Total seconds elapsed: {lastSeconds}. Inserted doc count: {lastDocumentCount}");

}

### Helper method to insert document ###

In [ ]:
public async Task InsertDocument(int taskId, Container container, int numberOfItemsToInsert) {

    requestUnitsConsumed[taskId] = 0; 

    for (var i = 0; i < numberOfItemsToInsert; i++) 
    {
        await Task.Delay(TimeSpan.FromMilliseconds(delayBetweenOperationsInMs)); // Wait some time on client side between each insert. 

        var newItem = new TestItem();
        var partitionKey = Guid.NewGuid().ToString();
        newItem.id = partitionKey; 
        newItem.val = Guid.NewGuid().ToString(); 

        var itemResponse = await container.CreateItemAsync(newItem, new PartitionKey(partitionKey));
        
        requestUnitsConsumed[taskId] += itemResponse.RequestCharge; // Keep track of how many RU/s have been consumed for this task
        Interlocked.Increment(ref documentsInserted); // Increment # doc inserted

    }
    Interlocked.Decrement(ref pendingTaskCount); // Consider task as completed when all documents have been inserted
    //Display.AsMarkdown($"Finished all inserts for taskId {taskId}. TotalDocsInserted so far: {documentsInserted}");

}

### DEMO: Change the delay between operations, to increase or decrease workload traffic

In [ ]:
// Reset the state
taskCount = 2;
pendingTaskCount = taskCount;
documentsInserted = 0;

delayBetweenOperationsInMs = 0; // Change this to simulate different workload patterns

Display.AsMarkdown($"Wait time between inserts is {delayBetweenOperationsInMs} ms.");

In [ ]:
var tasks = new List<Task>();
tasks.Add(LogOutputStatsSimple()); // Add a task that prints out average RU/s consumed every second. Runs until all inserts are completed. 

for (var j = 0; j < taskCount; j++)
{
    tasks.Add(InsertDocument(j, container, numberOfItemsToInsertPerTask));
}